In [15]:
import numpy as np
import torch
import torch.nn as nn
import math
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
pi = math.pi


In [4]:
def sign(x):
    y = torch.zeros(x.size())
    y[x > 0] = 1
    return y

In [5]:
class Mynet(nn.Module):
    def __init__(self, n):
        super(Mynet, self).__init__()
        self.n = n
        self.fc1 = nn.Linear(n, 1, bias=False)
        self.delta = Parameter(torch.Tensor(1))
        
    def forward(self, w):
        a = tocosines(w, self.n)
        r = self.fc1(a)
        out = torch.cat((r, torch.ones(r.size(0),1)*self.delta),1)
        return out

In [6]:
def tocosines(w, n):
    '''
    w: batch*1
    '''
    t = torch.arange(0, n, 1.0)
    c = torch.ones(1, n)*2
    c[0,0] = 1
    out = torch.cos(torch.ger(w, t))*c
    return out #batch*n

In [12]:
def customized_loss(w, out, alpha, ws, wp, tradeoff):
    '''
    w: batch * n
    out: batch * 2
    
    '''
    R = out[:,0].reshape(-1, 1)
    delta = out[:,1].reshape(-1, 1)
    v = torch.cat((R-alpha**2, 1/(alpha**2)-R, R-delta, -R), 1)
    v = F.relu(v)
#     print(v)
    reg = reg_lambda(w, ws, wp, tradeoff)
#     print (reg)
    loss = (torch.sum(v*reg) + torch.sum(delta))/w.size(0)
    return loss

In [16]:
def reg_lambda(w, ws, wp, tradeoff):
    reg = torch.zeros(w.size(0), 4)
    ind1 = ((w < wp).nonzero())
    ind2 = ((w > ws).nonzero())
    print (ind1)
    reg[ind1, 0] = 1
    reg[ind1, 1] = 1
    reg[ind2, 2] = 1
    reg[:, 3] = 1
    return reg*tradeoff #batch*4

In [17]:
class MyDataset(Dataset):
    def __init__(self, W):
        self.W = W

    def __len__(self):
        return len(self.W)

    def __getitem__(self, index):
        w = self.W[index]
        return w

In [18]:
alpha = 1.1
wp = 0.12*pi
ws = 0.24*pi
tradeoff = 20
n = 40
m = n*15
W = np.linspace(0, pi, m).astype('float32')
W = torch.from_numpy(W)

In [19]:
batch_size = 200
train_loader = torch.utils.data.DataLoader(
    MyDataset(W),
    batch_size=batch_size, shuffle=True)

In [20]:
model = Mynet(n)
optimizer = optim.SGD(model.parameters(), lr=0.00001, momentum=0.9)
scheduler = MultiStepLR(optimizer, milestones=[400, 600], gamma=0.2)
epochs = 1000

model_best = model
previous_loss = 10000
for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    scheduler.step(epoch)
    for i, w in enumerate(train_loader, 0):

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        out = model(w)
        loss = customized_loss(w, out, alpha, ws, wp, tradeoff)
        break
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('epoch = %d, loss = %.6f' %(epoch, running_loss/i))
    
    if running_loss < previous_loss:
        model_best = model
    
    previous_loss = running_loss
    
print('Finished Training')

tensor([[  5],
        [  8],
        [ 14],
        [ 28],
        [ 40],
        [ 52],
        [ 54],
        [ 55],
        [ 94],
        [117],
        [123],
        [133],
        [136],
        [155],
        [158],
        [164],
        [167],
        [169],
        [174]])


ZeroDivisionError: float division by zero

In [1]:
def get_result():
    a =[]
    for parameter in model_best.parameters():
        a.append(parameter.data)
    delta = a[0]
    r = torch.mm(a[1].permute(1, 0), a[2].permute(1, 0))
    return delta, r.reshape(1, -1)

In [2]:
delta, r = get_result()
print(delta)
print (r)

NameError: name 'model_best' is not defined

In [ ]:
test_loader = torch.utils.data.DataLoader(
    MyDataset(W),
    batch_size=20, shuffle=True)
with torch.no_grad():
    for w in test_loader:
        print(w)
        out = model(w)
        print (out)
        R = out[:,0].reshape(-1, 1)
        delta = out[:,1].reshape(-1, 1)
        v = torch.cat((R-alpha**2, 1/(alpha**2)-R, R-delta, -R), 1)
        print (v)
        v = F.relu(v)
        print(v)
        reg = reg_lambda(w, ws, wp, tradeoff)
        print (reg)
        print (v*reg)
        loss = (torch.sum(v*reg) + torch.sum(delta))/w.size(0)
        loss = customized_loss(w, out, alpha, ws, wp, tradeoff)
        break